In [1]:
import pandas as pd
import os
import datetime
from lockdown import LockDown

In [2]:
all_data = []
dtype = {
    "country_region_code":str,
    "country_region":str,
    "sub_region_1":str,
    "sub_region_2":str,
    "metro_area":str,
    "iso_3166_2_code":str,
    "census_fips_code":str,
    "place_id":str,
    "date":object,
    "retail_and_recreation_percent_change_from_baseline":float,
    "grocery_and_pharmacy_percent_change_from_baseline":float,
    "parks_percent_change_from_baseline":float,
    "transit_stations_percent_change_from_baseline":float,
    "workplaces_percent_change_from_baseline":float,
    "residential_percent_change_from_baseline":float
}
deleted_countries = [
    "LT",#Letonya
    "LU",#Lüksemburg
    "MT",#Malta
    "SI",#Slovenya
    "SE",#İsveç
]

dropped_columns = [
    'sub_region_1',
    'sub_region_2',
    'metro_area',
    'iso_3166_2_code',
    'census_fips_code',
    'place_id',
]

new_column_names = {
    'retail_and_recreation_percent_change_from_baseline': 'retail_and_recreation',
    'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_and_pharmacy',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline': 'transit_stations',
    'workplaces_percent_change_from_baseline': 'workplaces',
    'residential_percent_change_from_baseline': 'residential',
}

retail_and_recreation_factor = -0.25
grocery_and_pharmacy_factor = -0.05
parks_factor = -0.1
transit_stations_factor = -0.1
workplaces_factor = -0.2
residential_factor = 0.3

lockdowns_data = [
    ['AT', '16.03.2020', '13.04.2020'],
    ['AT', '03.11.2020', '30.11.2020'],
    ['AT', '26.12.2020', '07.02.2021'],
    ['AT', '22.11.2021', '12.12.2021'],
    ['BE', '18.03.2020', '04.05.2020'],
    ['BE', '02.11.2020', '14.12.2020'],
    ['BE', '27.03.2021', '26.04.2021'],
    ['BG', '13.03.2020', '15.06.2020'],
    ['BG', '28.11.2020', '31.01.2021'],
    ['BG', '22.03.2021', '31.03.2021'],
    ['HR', '18.03.2020', '11.05.2020'],
    ['HR', '22.12.2020', '29.12.2020'],
    ['CZ', '16.03.2020', '12.04.2020'],
    ['CZ', '22.10.2020', '28.03.2021'],
    ['DK', '12.03.2020', '13.04.2020'],
    ['DK', '25.12.2020', '01.03.2021'],
    ['FR', '17.03.2020', '11.05.2020'],
    ['FR', '30.10.2020', '15.12.2020'],
    ['FR', '04.04.2021', '03.05.2021'],
    ['FI', '16.03.2020', '13.05.2020'],
    ['FI', '08.03.2021', '28.03.2021'],
    ['EE', '11.03.2021', '11.04.2021'],
    ['DE', '16.03.2020', '30.05.2020'],
    ['DE', '02.11.2020', '01.03.2021'],
    ['GR', '23.03.2020', '04.05.2020'],
    ['GR', '07.11.2020', '22.03.2021'],
    ['HU', '28.03.2020', '10.04.2020'],
    ['IE', '12.03.2020', '18.05.2020'],
    ['IE', '21.10.2020', '01.12.2020'],
    ['IE', '24.12.2020', '12.04.2021'],
    ['IT', '09.03.2020', '12.04.2021'],
    ['IT', '24.12.2020', '06.01.2021'],
    ['IT', '15.03.2021', '30.04.2021'],
    ['LT', '16.03.2020', '18.06.2020'],
    ['LT', '07.11.2020', '01.07.2021'],
    ['NL', '15.03.2020', '06.04.2020'],
    ['NL', '15.12.2020', '05.06.2021'],
    ['NL', '13.11.2021', '12.12.2021'],
    ['PL', '13.03.2020', '11.04.2020'],
    ['PL', '28.12.2020', '17.01.2021'],
    ['PL', '20.03.2021', '25.04.2021'],
    ['PT', '19.03.2020', '02.04.2020'],
    ['PT', '15.01.2021', '15.03.2021'],
    ['RO', '25.03.2020', '12.05.2020'],
    ['SI', '25.11.2021', '09.12.2021'],
    ['ES', '14.03.2020', '09.05.2020'],
]

columns_will_get_from_who_data = ['New_cases', 'New_deaths']

files = os.listdir("assets")
countries = [x for x in files if x[5:7] not in deleted_countries]  # deleted unnecessary countries

def sort_func(string):
    return string[5:7] + string[:5]

countries = sorted(countries, key=sort_func)

who_data = pd.read_csv("WHO-COVID-19-global-data.csv")
who_data['date'] = pd.to_datetime(who_data['Date_reported'])

In [3]:
for csv_file in countries:
    file_name = csv_file.split(".")[0]
    data_frame = pd.read_csv(
        "assets/{filename}".format(filename=csv_file),
        dtype=dtype
    )
    all_data.append(data_frame)

In [4]:
summed_datas = []
for x1, x2 in zip(all_data[::2], all_data[1::2]):
    summed_data = pd.concat([x1, x2])
    summed_datas.append(summed_data)

In [5]:
clean_all_data = []
for df in summed_datas:
    cleaned_data = df[df['sub_region_1'].isnull()]
    clean_all_data.append(cleaned_data)

In [6]:
summed_data_frame = pd.concat(clean_all_data, ignore_index=True)
summed_data_frame['date'] = pd.to_datetime(summed_data_frame['date'])
summed_data_frame.drop(columns=dropped_columns, inplace=True)
summed_data_frame.rename(columns=new_column_names, inplace=True)

lockdowns = [ LockDown(x, y, z) for x,y,z in lockdowns_data]
for x in lockdowns:
    x.set_data(summed_data_frame, who_data, columns_will_get_from_who_data)

In [7]:
summed_data_frame.to_csv("data.csv")

In [8]:
max_retail_and_recreation = summed_data_frame['retail_and_recreation'].max()
max_grocery_and_pharmacy = summed_data_frame['grocery_and_pharmacy'].max()
max_parks = summed_data_frame['parks'].max()
max_transit_stations = summed_data_frame['transit_stations'].max()
max_workplaces = summed_data_frame['workplaces'].max()
max_residential = summed_data_frame['residential'].max()

In [9]:
min_retail_and_recreation = summed_data_frame['retail_and_recreation'].min()
min_grocery_and_pharmacy = summed_data_frame['grocery_and_pharmacy'].min()
min_parks = summed_data_frame['parks'].min()
min_transit_stations = summed_data_frame['transit_stations'].min()
min_workplaces = summed_data_frame['workplaces'].min()
min_residential = summed_data_frame['residential'].min()

In [10]:
for lockdown in lockdowns:
    avg_values = lockdown.get_avg_values()
    success_percent_retail_and_recreation = (max_retail_and_recreation - avg_values['retail_and_recreation']) / (max_retail_and_recreation - min_retail_and_recreation)
    success_percent_grocery_and_pharmacy = (max_grocery_and_pharmacy - avg_values['grocery_and_pharmacy']) / (max_grocery_and_pharmacy - min_grocery_and_pharmacy)
    success_percent_parks = (max_parks - avg_values['parks']) / (max_parks - min_parks)
    success_percent_transit_stations = (max_transit_stations - avg_values['transit_stations']) / (max_transit_stations - min_transit_stations)
    success_percent_workplaces = (max_workplaces - avg_values['workplaces']) / (max_workplaces - min_workplaces)
    success_percent_residential = (avg_values['residential'] - min_residential) / (max_residential - min_residential)
    success_point_for_lockdown = 0
    success_point_for_lockdown += success_percent_retail_and_recreation * retail_and_recreation_factor
    success_point_for_lockdown += success_percent_grocery_and_pharmacy * grocery_and_pharmacy_factor
    success_point_for_lockdown += success_percent_parks * parks_factor
    success_point_for_lockdown += success_percent_transit_stations * transit_stations_factor
    success_point_for_lockdown += success_percent_workplaces * workplaces_factor
    success_point_for_lockdown += success_percent_residential * retail_and_recreation_factor
    lockdown.set_lockdown_success_point(success_point_for_lockdown)

    avg_new_cases_before_lockdown = lockdown.get_before_lockdown_average_case_count_by_week(who_data, 2)
    avg_new_cases_after_lockdown = lockdown.get_after_lockdown_average_case_count_by_week(who_data, 1)
    change_on_percent = (avg_new_cases_after_lockdown - avg_new_cases_before_lockdown) / avg_new_cases_after_lockdown